In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

In [2]:
test = pd.read_csv('include/test.csv')
train = pd.read_csv('include/train.csv').dropna()

In [3]:
def substrings_in_string(big_string, substrings):
    for substring in substrings:
        if str(big_string).find(substring) != -1:
            return substring
    return ''

In [4]:
train_reformated = train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
cabin_list = ['A', 'B', 'C', 'D', 'E', 'F', 'T', 'G', 'Unknown']
train_reformated['Deck'] = train['Cabin'].map(lambda x: substrings_in_string(x, cabin_list))
train_reformated['Family_Size'] = train_reformated['SibSp'] + train_reformated['Parch']
train_reformated['Fare_Per_Person'] = train_reformated['Fare'] /(train_reformated['Family_Size'] + 1)

train_reformated['Sex'] =  LabelEncoder().fit_transform(train_reformated['Sex'])
train_reformated['Embarked'] = LabelEncoder().fit_transform(train_reformated['Embarked'])
train_reformated['Deck'] = LabelEncoder().fit_transform(train_reformated['Deck'])
print(train_reformated.head())

    Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  Deck  \
1          1       1    0  38.0      1      0  71.2833         0     2   
3          1       1    0  35.0      1      0  53.1000         2     2   
6          0       1    1  54.0      0      0  51.8625         2     4   
10         1       3    0   4.0      1      1  16.7000         2     6   
11         1       1    0  58.0      0      0  26.5500         2     2   

    Family_Size  Fare_Per_Person  
1             1        35.641650  
3             1        26.550000  
6             0        51.862500  
10            2         5.566667  
11            0        26.550000  


In [5]:
clf = RandomForestClassifier()
clf.fit(train_reformated.loc[:, train_reformated.columns != 'Survived'], train_reformated['Survived'])
print(clf.feature_importances_)

[ 0.00836482  0.24538938  0.1583901   0.02115705  0.01052946  0.23505852
  0.01237421  0.11293827  0.03005173  0.16574646]


In [6]:
test_passengerId = test['PassengerId']
test_reformated = test.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
cabin_list = ['A', 'B', 'C', 'D', 'E', 'F', 'T', 'G', 'Unknown']
test_reformated['Deck'] = test['Cabin'].map(lambda x: substrings_in_string(x, cabin_list))
test_reformated['Family_Size'] = test_reformated['SibSp'] + test_reformated['Parch']
test_reformated['Fare_Per_Person'] = test_reformated['Fare'] /(test_reformated['Family_Size'] + 1)

test_reformated['Sex'] =  LabelEncoder().fit_transform(test_reformated['Sex'])
test_reformated['Embarked'] = LabelEncoder().fit_transform(test_reformated['Embarked'])
test_reformated['Deck'] = LabelEncoder().fit_transform(test_reformated['Deck'])

In [7]:
print(test_reformated.columns[test_reformated.isnull().any()].tolist())
test_reformated = test_reformated.fillna(test_reformated.mean())

test_predictions = clf.predict(test_reformated).reshape(-1, 1)

combined = np.column_stack((test_passengerId, test_predictions))
df = pd.DataFrame(combined, columns=["PassengerId", "Survived"])

['Age', 'Fare', 'Fare_Per_Person']


In [8]:
df.to_csv('csv/answers_after_prepro.csv',index=False)